In [1]:
import warnings
from pathlib import Path
import subprocess
import numpy as np
from MDAnalysis import Universe
from openbabel import pybel

# Filter warnings
warnings.filterwarnings("ignore")

class Structure(Universe):
    """Core object to load structures with."""
    
    @classmethod
    def from_string(cls, pdb_path):
        """Load a structure from a local PDB file."""
        return cls(pdb_path)

# Define paths
HERE = Path.cwd()
DATA = HERE / "data"
DATA.mkdir(exist_ok=True)

# Load local structure
pdb_path = DATA / "2ito.pdb"  # Ensure this points to your local PDB file
structure = Structure.from_string(pdb_path)

# Prepare protein file
protein_path = DATA / "protein.pdb"
protein = structure.select_atoms("protein")
protein.write(str(protein_path))

def pdb_to_pdbqt(pdb_path, pdbqt_path, pH=7.4):
    """Convert a PDB file to a PDBQT file."""
    molecule = list(pybel.readfile("pdb", str(pdb_path)))[0]
    molecule.OBMol.CorrectForPH(pH)
    molecule.addh()
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()
    molecule.write("pdbqt", str(pdbqt_path), overwrite=True)

# Convert protein to PDBQT format
protein_pdbqt_path = DATA / "protein.pdbqt"
pdb_to_pdbqt(protein_path, protein_pdbqt_path)

# Define ligand residue name and calculate pocket parameters
ligand_resname = "IRE"  # https://proteins.plus/ 需要 在这里找到 配体
ligand = structure.select_atoms(f"resname {ligand_resname}")

# Calculate pocket center and size
pocket_center = (ligand.positions.max(axis=0) + ligand.positions.min(axis=0)) / 2
pocket_size = ligand.positions.max(axis=0) - ligand.positions.min(axis=0) + 5

# Define ligand SMILES and convert to PDBQT
smiles = "COC1=C(C=C2C(=C1)N=CN=C2NC3=CC(=C(C=C3)F)Cl)OCCCN4CCOCC4"
ligand_path = DATA / "ligand.pdbqt"

def smiles_to_pdbqt(smiles, pdbqt_path, pH=7.4):
    """Convert a SMILES string to a PDBQT file."""
    molecule = pybel.readstring("smi", smiles)
    molecule.OBMol.CorrectForPH(pH)
    molecule.addh()
    molecule.make3D(forcefield="mmff94s", steps=10000)
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()
    molecule.write("pdbqt", str(pdbqt_path), overwrite=True)

# Convert ligand SMILES to PDBQT format
smiles_to_pdbqt(smiles, ligand_path)

# Run Smina docking
def run_smina(ligand_path, protein_path, out_path, pocket_center, pocket_size):
    """Perform docking with Smina."""
    output_text = subprocess.check_output(
        [
            "smina",
            "--ligand", str(ligand_path),
            "--receptor", str(protein_path),
            "--center_x", str(pocket_center[0]),
            "--center_y", str(pocket_center[1]),
            "--center_z", str(pocket_center[2]),
            "--size_x", str(pocket_size[0]),
            "--size_y", str(pocket_size[1]),
            "--size_z", str(pocket_size[2]),
            "--out", str(out_path)
        ]
    )
    print(output_text.decode())

# 保存output_text结果
def save_output_text(output_text, output_path):
    """Save Smina output text to a file."""
    with open(output_path, "w") as f:
        f.write(output_text)


# Define output path for docking results
docking_output_path = DATA / "docking_results.pdbqt"

# Run the docking
run_smina(ligand_path, protein_pdbqt_path, docking_output_path, pocket_center, pocket_size)


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /Users/wangyang/Desktop/teachopencadd/test/data/protein.pdb)



   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: -1512338160

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------

In [2]:
import warnings  # 导入警告模块
from pathlib import Path  # 导入路径处理模块
import subprocess  # 导入子进程模块
import numpy as np  # 导入NumPy模块
from MDAnalysis import Universe  # 从MDAnalysis导入Universe类
from openbabel import pybel  # 从Open Babel导入pybel模块
import nglview as nv  # 导入nglview用于可视化

import openmm  # 导入OpenMM模块
import openmm.app  # 导入OpenMM.app模块

# Filter warnings
warnings.filterwarnings("ignore")  # 忽略警告信息

class Structure(Universe):
    """Core object to load structures with."""
    
    @classmethod
    def from_string(cls, pdb_path):
        """Load a structure from a local PDB file."""
        return cls(pdb_path)  # 从本地PDB文件加载结构

# Define paths
HERE = Path.cwd()  # 获取当前工作目录
DATA = HERE / "data"  # 定义数据目录
DATA.mkdir(exist_ok=True)  # 如果目录不存在则创建

# Load local structure
pdb_path = DATA / "2ito.pdb"  # 确保这个路径指向本地的PDB文件
structure = Structure.from_string(pdb_path)  # 加载本地结构

# Prepare protein file
protein_path = DATA / "protein.pdb"  # 定义蛋白质文件路径
protein = structure.select_atoms("protein")  # 选择蛋白质原子
protein.write(str(protein_path))  # 将蛋白质写入文件

def pdb_to_pdbqt(pdb_path, pdbqt_path, pH=7.4):
    """Convert a PDB file to a PDBQT file."""
    molecule = list(pybel.readfile("pdb", str(pdb_path)))[0]  # 读取PDB文件
    molecule.OBMol.CorrectForPH(pH)  # 根据pH值进行修正
    molecule.addh()  # 添加氢原子
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()  # 获取部分电荷
    molecule.write("pdbqt", str(pdbqt_path), overwrite=True)  # 写入PDBQT文件

# Convert protein to PDBQT format
protein_pdbqt_path = DATA / "protein.pdbqt"  # 定义转换后的蛋白质PDBQT文件路径
pdb_to_pdbqt(protein_path, protein_pdbqt_path)  # 转换蛋白质为PDBQT格式

# Define ligand residue name and calculate pocket parameters
ligand_resname = "IRE"  # https://proteins.plus/ 需要 在这里找到 配体
ligand = structure.select_atoms(f"resname {ligand_resname}")  # 选择配体原子

# Calculate pocket center and size
pocket_center = (ligand.positions.max(axis=0) + ligand.positions.min(axis=0)) / 2  # 计算口袋中心
pocket_size = ligand.positions.max(axis=0) - ligand.positions.min(axis=0) + 5  # 计算口袋大小

# Define ligand SMILES and convert to PDBQT
smiles = "COC1=C(C=C2C(=C1)N=CN=C2NC3=CC(=C(C=C3)F)Cl)OCCCN4CCOCC4"  # 定义配体SMILES
ligand_path = DATA / "ligand.pdbqt"  # 定义配体PDBQT文件路径

def smiles_to_pdbqt(smiles, pdbqt_path, pH=7.4):
    """Convert a SMILES string to a PDBQT file."""
    molecule = pybel.readstring("smi", smiles)  # 从SMILES字符串读取分子
    molecule.OBMol.CorrectForPH(pH)  # 根据pH值进行修正
    molecule.addh()  # 添加氢原子
    molecule.make3D(forcefield="mmff94s", steps=10000)  # 生成3D构象
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()  # 获取部分电荷
    molecule.write("pdbqt", str(pdbqt_path), overwrite=True)  # 写入PDBQT文件

# Convert ligand SMILES to PDBQT format
smiles_to_pdbqt(smiles, ligand_path)  # 将配体SMILES转换为PDBQT格式

# Run Smina docking
def run_smina(ligand_path, protein_path, out_path, pocket_center, pocket_size):
    """Perform docking with Smina."""
    output_text = subprocess.check_output([
        "smina",  # Smina命令
        "--receptor", str(protein_path),  # 蛋白质路径
        "--ligand", str(ligand_path),  # 配体路径
        "--out", str(out_path),  # 输出路径
        "--center_x", str(pocket_center[0]),  # 口袋中心x坐标
        "--center_y", str(pocket_center[1]),  # 口袋中心y坐标
        "--center_z", str(pocket_center[2]),  # 口袋中心z坐标
        "--size_x", str(pocket_size[0]),  # 口袋大小x
        "--size_y", str(pocket_size[1]),  # 口袋大小y
        "--size_z", str(pocket_size[2])   # 口袋大小z
    ])
    print(output_text.decode())
    # 保存output_text结果
    # 报错.txt文件，保存到DATA目录下
    with open(DATA / "output_text.txt", "w") as f:
        f.write(output_text.decode())
    return output_text.decode("utf-8")  # 返回输出文本

# 执行分子对接
docking_out_path = DATA / "docking_poses.sdf"  # 定义对接输出文件路径
run_smina(ligand_path, protein_pdbqt_path, docking_out_path, pocket_center, pocket_size)  # 运行对接






*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /Users/wangyang/Desktop/teachopencadd/test/data/protein.pdb)



   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: 1152005504

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------


'   _______  _______ _________ _        _______ \n  (  ____ \\(       )\\__   __/( (    /|(  ___  )\n  | (    \\/| () () |   ) (   |  \\  ( || (   ) |\n  | (_____ | || || |   | |   |   \\ | || (___) |\n  (_____  )| |(_)| |   | |   | (\\ \\) ||  ___  |\n        ) || |   | |   | |   | | \\   || (   ) |\n  /\\____) || )   ( |___) (___| )  \\  || )   ( |\n  \\_______)|/     \\|\\_______/|/    )_)|/     \\|\n\n\nsmina is based off AutoDock Vina. Please cite appropriately.\n\nWeights      Terms\n-0.035579    gauss(o=0,_w=0.5,_c=8)\n-0.005156    gauss(o=3,_w=2,_c=8)\n0.840245     repulsion(o=0,_c=8)\n-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)\n-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)\n1.923        num_tors_div\n\nUsing random seed: 1152005504\n\n0%   10   20   30   40   50   60   70   80   90   100%\n|----|----|----|----|----|----|----|----|----|----|\n***************************************************\n\nmode |   affinity | dist from best mode\n     | (kcal/mol) | rmsd l.b.| rmsd u.b.

In [3]:
def split_sdf_file(sdf_path):
    """
    Split an SDF file into separate files for each molecule.
    Each file is named with consecutive numbers.

    Parameters
    ----------
    sdf_path: str or pathlib.Path
        Path to SDF file that should be split.
    """
    sdf_path = Path(sdf_path)  # 转换为Path对象
    stem = sdf_path.stem  # 获取文件名（不带扩展名）
    parent = sdf_path.parent  # 获取父目录
    molecules = pybel.readfile("sdf", str(sdf_path))  # 读取SDF文件中的分子
    for i, molecule in enumerate(molecules, 1):  # 遍历每个分子
        molecule.write("sdf", str(parent / f"{stem}_{i}.sdf"), overwrite=True)  # 写入单独的SDF文件
    return

# 分割对接输出的SDF文件
split_sdf_file(DATA / "docking_poses.sdf")  # 调用函数分割SDF文件

In [4]:
# 可视化对接结果
docking_pose_id = 1  # 定义要查看的对接位点ID
view = nv.show_structure_file(  # 创建可视化对象
    str(DATA / f"docking_poses_{docking_pose_id}.sdf"),  # 指定要显示的SDF文件
    representations=[{"params": {}, "type": "licorice"}],  # 设置可视化参数
)
view.add_pdbid(pdb_path.stem)  # 添加PDB ID到视图
view  # 显示视图



NGLWidget()

In [5]:
view.render_image(trim=True, factor=2)  # 渲染图像并设置缩放因子


Image(value=b'', width='99%')

In [6]:
view._display_image()  # 显示渲染的图像

## 全自动分子对接

In [14]:
import warnings  # 导入警告模块
from pathlib import Path  # 导入路径处理模块
import subprocess  # 导入子进程模块
import numpy as np  # 导入NumPy模块
from MDAnalysis import Universe  # 从MDAnalysis导入Universe类
from openbabel import pybel  # 从Open Babel导入pybel模块
import nglview as nv  # 导入nglview用于可视化

# Filter warnings
warnings.filterwarnings("ignore")  # 忽略警告信息

class Structure(Universe):
    """Core object to load structures with."""
    
    @classmethod
    def from_string(cls, pdb_path):
        """Load a structure from a local PDB file."""
        return cls(pdb_path)  # 从本地PDB文件加载结构

# Define paths
HERE = Path.cwd()  # 获取当前工作目录
DATA = HERE / "data"  # 定义数据目录
DATA.mkdir(exist_ok=True)  # 如果目录不存在则创建

# Load local structure
pdb_path = DATA / "2ito.pdb"  # 确保这个路径指向本地的PDB文件
structure = Structure.from_string(pdb_path)  # 加载本地结构

# Prepare protein file
protein_path = DATA / "protein.pdb"  # 定义蛋白质文件路径
protein = structure.select_atoms("protein")  # 选择蛋白质原子
protein.write(str(protein_path))  # 将蛋白质写入文件

def pdb_to_pdbqt(pdb_path, pdbqt_path, pH=7.4):
    """Convert a PDB file to a PDBQT file."""
    molecule = list(pybel.readfile("pdb", str(pdb_path)))[0]  # 读取PDB文件
    molecule.OBMol.CorrectForPH(pH)  # 根据pH值进行修正
    molecule.addh()  # 添加氢原子
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()  # 获取部分电荷
    molecule.write("pdbqt", str(pdbqt_path), overwrite=True)  # 写入PDBQT文件

# Convert protein to PDBQT format
protein_pdbqt_path = DATA / "protein.pdbqt"  # 定义转换后的蛋白质PDBQT文件路径
pdb_to_pdbqt(protein_path, protein_pdbqt_path)  # 转换蛋白质为PDBQT格式

# Define ligand residue name and calculate pocket parameters
ligand_resname = "IRE"  # 确保与配体的残基名称匹配
ligand = structure.select_atoms(f"resname {ligand_resname}")  # 选择配体原子

# Calculate pocket center and size
pocket_center = (ligand.positions.max(axis=0) + ligand.positions.min(axis=0)) / 2  # 计算口袋中心
pocket_size = ligand.positions.max(axis=0) - ligand.positions.min(axis=0) + 5  # 计算口袋大小

# Define multiple ligand SMILES
smiles_list = [
    "CC(C)S(=O)(=O)c1ccccc1Nc1nc(Nc2ccc(NC(=O)CN3CCC(C(=O)O)CC3)cc2)ncc1Cl",
    "CC(C)S(=O)(=O)c1ccccc1Nc1nc(Nc2ccc(CN3CCC(C(=O)NCCO)CC3)cc2)ncc1Cl",
    "CC(C)S(=O)(=O)c1ccccc1Nc1nc(Nc2ccc(OCC(=O)N3CCOCC3)cc2)ncc1Cl",
    "CN1CCc2cc(Nc3ncc(C(F)(F)F)c(CCc4ccccc4CC(N)=O)n3)ccc2C1",
    "CC(=O)N1CCC(c2ccc(Nc3ncc(C(F)(F)F)c(CCc4ccccc4CC(N)=O)n3)cc2)CC1",
    "CN(c1ccccc1CNc1nc(Nc2ccc3c(c2)CNC3=O)ncc1C(F)(F)F)S(C)(=O)=O",
    "CNC(=O)c1ccc(Nc2ncc(C(F)(F)F)c(NCc3nccnc3N(C)S(C)(=O)=O)n2)cc1",
    "CNC(=O)c1ccc(Nc2ncc(C(F)(F)F)c(NCc3cccnc3N(C)S(C)(=O)=O)n2)cc1",
    "CN(c1ccccc1CNc1nc(Nc2ccc(S(C)(=O)=O)cc2)ncc1C(F)(F)F)S(C)(=O)=O",
    "COc1cc(C(=O)N[C@H]2CCN(C)C[C@H]2F)c(C)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)NC2CCN(C)CC2)c(F)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)[C@@H]3C)n1",
    "COc1cc(C(=O)NC2CCN(CCO)CC2)ccc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)N[C@@H]2CCN(C)C[C@@H]2OC)c(C)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)C3(C)C)n1",
    "COc1cc(C(=O)NC2CCN(C3CCOCC3)CC2)c(C)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)N[C@H]2CCN(C)C[C@H]2OC)c(Cl)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)[C@H]3C)n1",
    "COc1cc(C(=O)NC2CCN(C)CC2)c(F)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)[C@H]3C)n1",
    "COc1cc(C(=O)N[C@H]2CCN(C)C[C@H]2F)c(F)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(OC)C3)n1",
    "COc1cc(C(=O)N[C@@H]2CCN(C)C[C@@H]2OC)c(Cl)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)[C@@H]3C)n1",
    "COc1cc(C(=O)N[C@@H]2CCN(C)C[C@@H]2OC)c(F)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)N[C@H]2CCN(C)C[C@H]2F)c(F)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)[C@H]3C)n1",
    "COc1cc(C(=O)N[C@@H]2CCN(C)C[C@@H]2OC)c(F)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)[C@@H]3C)n1",
    "COc1cc(C(=O)N[C@H]2CCN(C)C[C@H]2F)c(F)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)[C@@H]3C)n1",
    "COc1cc(C(=O)NC2CCN(C)CC2)ccc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)N[C@@H]2CCN(C)C[C@@H]2OC)c(F)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)[C@H]3C)n1",
    "COc1cc(C(=O)N[C@@H]2CCN(C)C[C@@H]2OC)c(F)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(OC)C3)n1",
    "COc1cc(C(=O)N[C@@H]2CCCN(C)C2)c(F)cc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(OC)C3)n1",
    "COc1cc(C(=O)N[C@@H]2CCCN(C)C2)ccc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(OC)C3)n1",
    "COc1cc(O[C@@H]2CO[C@H]3[C@@H]2OC[C@H]3O)ccc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3C)n1",
    "COc1cc(C(=O)N[C@@H]2CCCN(C)C2)c(C)cc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(N2CCN(C)CC2)ccc1Nc1ncc(C#N)c(Oc2cccc3c2C(=O)CC3)n1",
    "COc1cc(C(=O)NC2CCN(C)CC2)c(Cl)cc1Nc1ncc(C#N)c(Oc2cccc3c2C(=O)CC3)n1",
    "COc1cc(C(=O)NC2CCCN(C)C2)c(C)cc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)NC2CCN(C)CC2)c(F)cc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(OC)C3)n1",
    "COc1cc(C(=O)N[C@@H]2CCN(C)C[C@@H]2OC)c(Cl)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "CN(c1ccccc1CNc1nc(Nc2ccc3c(c2)CS(=O)(=O)N3)ncc1C(F)(F)F)S(C)(=O)=O",
    "COc1cc(C(=O)N[C@H]2CCN(C)C[C@H]2F)c(Cl)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)NC[C@H](O)CO)c(F)cc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)NCCN(C)C)c(F)cc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)N[C@@H]2CCN(C)C[C@@H]2OC)c(F)cc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(N2CCN(C)CC2)ccc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)N[C@H]2CCCN(C)C2)c(C)cc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)NCCN(C)C)c(C)cc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)NC2CCN(C)CC2)c(C)cc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "CNC(=O)c1cc(OC)c(Nc2ncc(C(F)(F)F)c(Oc3cccc4c3C(=O)N(C)C4)n2)cc1C",
    "COc1cc(C(=O)N[C@@H]2CCN(C)C[C@@H]2F)ccc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)N[C@H]2CCN(C)C[C@H]2F)c(F)cc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)NC2CCN(C)CC2)c(F)cc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)N[C@H]2CCN(C)C[C@H]2F)ccc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)NC2CCN(C)CC2)ccc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)NC2CCN(C)CC2)c(F)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(OC)C3)n1",
    "COc1cc(C(=O)N[C@H]2CCN(C)C[C@H]2F)ccc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3C)n1",
    "COc1cc(N2CCN(C)CC2)ccc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)C3C)n1",
    "COc1cc(C(=O)NC2CCN(C)CC2)c(Cl)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(CN(C)C)ccc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)NC2CCN(C)CC2)c(C)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)N[C@@H]2CCN(C)C[C@@H]2OC)c(C)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COCCN1CCC(NC(=O)c2ccc(Nc3ncc(C(F)(F)F)c(Oc4cccc5c4C(=O)N(C)C5)n3)c(OC)c2)CC1",
    "COc1cc(C[C@@H](O)CO)ccc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)N[C@@H]2CCN(C)C[C@H]2OC)c(C)cc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)N[C@H]2CCN(C)C[C@@H]2OC)c(C)cc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)NC2CCN(C)C[C@H]2OC)ccc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)N[C@H]2CCN(C)C[C@@H]2OC)c(F)cc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "Cc1cc(C(=O)NC2CCN(C)CC2)ccc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "Cc1cc(C(=O)NC2CCCN(C)C2)ccc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "CNC(=O)c1ccccc1Nc1nc(Nc2ccc3c(c2OC)CCC[C@@H](N2CCN(CCO)CC2)C3)ncc1Cl",
    "COc1cc(C[C@H](O)CO)ccc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(N2CCN(CC(=O)N(C)C)CC2)ccc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COc1cc(C(=O)NC2CCN(C)C[C@H]2OC)c(C)cc1Nc1ncc(Cl)c(Oc2cccc3c2C(=O)N(C)C3(C)C)n1",
    "Cc1cc(C(=O)NC2CCN(CCO)CC2)ccc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
    "COCCN1CCC(NC(=O)c2ccc(Nc3ncc(C(F)(F)F)c(Oc4cccc5c4C(=O)N(C)C5)n3)c(C)c2)CC1",
    "COc1cc(C(=O)N[C@H]2CCN(C)C[C@@H]2OC)ccc1Nc1ncc(C(F)(F)F)c(Oc2cccc3c2C(=O)N(C)C3)n1",
]  # 可以添加更多SMILES字符串

# Convert ligands to PDBQT format
for idx, smiles in enumerate(smiles_list):
    ligand_path = DATA / f"ligand_{idx + 1}.pdbqt"  # 定义每个配体的PDBQT文件路径
    molecule = pybel.readstring("smi", smiles)  # 从SMILES字符串读取分子
    molecule.OBMol.CorrectForPH(7.4)  # 根据pH值进行修正
    molecule.addh()  # 添加氢原子
    molecule.make3D(forcefield="mmff94s", steps=10000)  # 生成3D结构
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()  # 获取部分电荷
    molecule.write("pdbqt", str(ligand_path), overwrite=True)  # 写入PDBQT文件

# Run Smina docking for each ligand
for idx in range(len(smiles_list)):
    ligand_path = DATA / f"ligand_{idx + 1}.pdbqt"  # 获取当前配体的路径
    out_path = DATA / f"docking_poses_{idx + 1}.sdf"  # 定义对接结果输出路径
    subprocess.run([  # 使用subprocess运行Smina
        "smina",
        "--ligand", str(ligand_path),
        "--receptor", str(protein_pdbqt_path),
        "--center_x", str(pocket_center[0]),
        "--center_y", str(pocket_center[1]),
        "--center_z", str(pocket_center[2]),
        "--size_x", str(pocket_size[0]),
        "--size_y", str(pocket_size[1]),
        "--size_z", str(pocket_size[2]),
        "--out", str(out_path),
        "--exhaustiveness", "8"
    ])

# Function to split SDF file
def split_sdf_file(sdf_path):
    """
    Split an SDF file into separate files for each molecule.
    Each file is named with consecutive numbers.

    Parameters
    ----------
    sdf_path: str or pathlib.Path
        Path to SDF file that should be split.
    """
    sdf_path = Path(sdf_path)  # 转换为Path对象
    stem = sdf_path.stem  # 获取文件名（不带扩展名）
    parent = sdf_path.parent  # 获取文件所在目录
    molecules = pybel.readfile("sdf", str(sdf_path))  # 读取SDF文件中的分子
    for i, molecule in enumerate(molecules, 1):
        molecule.write("sdf", str(parent / f"{stem}_{i}.sdf"), overwrite=True)  # 写入每个分子到单独的SDF文件
    return

# Split each docking poses SDF file
for idx in range(len(smiles_list)):
    split_sdf_file(DATA / f"docking_poses_{idx + 1}.sdf")  # 分割对接结果SDF文件

# Visualization (optional)
docking_pose_id = 1  # 选择要查看的对接结果编号
view = nv.show_structure_file(  # 显示结构文件
    str(DATA / f"docking_poses_{docking_pose_id}.sdf"),
    representations=[{"params": {}, "type": "licorice"}],
)
view.add_pdbid("2ito")  # 添加PDB ID以便可视化
view  # 显示可视化结果


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /Users/wangyang/Desktop/teachopencadd/test/data/protein.pdb)



   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: -620780717

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------


NGLWidget()

In [17]:
import warnings  # 导入警告模块
from pathlib import Path  # 导入路径处理模块
import subprocess  # 导入子进程模块
import numpy as np  # 导入NumPy模块
from MDAnalysis import Universe  # 从MDAnalysis导入Universe类
from openbabel import pybel  # 从Open Babel导入pybel模块
import nglview as nv  # 导入nglview用于可视化

# 忽略警告信息
warnings.filterwarnings("ignore")

class Structure(Universe):
    """用于加载结构的核心对象。"""

    @classmethod
    def from_string(cls, pdb_path):
        """从本地PDB文件加载结构。"""
        return cls(pdb_path)

# 定义路径
HERE = Path.cwd()
DATA = HERE / "data"
DATA.mkdir(exist_ok=True)

# 加载本地结构
pdb_path = DATA / "2ito.pdb"
structure = Structure.from_string(pdb_path)

# 准备蛋白质文件
protein_path = DATA / "protein.pdb"
protein = structure.select_atoms("protein")
protein.write(str(protein_path))

def pdb_to_pdbqt(pdb_path, pdbqt_path, pH=7.4):
    """将PDB文件转换为PDBQT文件。"""
    molecule = list(pybel.readfile("pdb", str(pdb_path)))[0]
    molecule.OBMol.CorrectForPH(pH)
    molecule.addh()
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()
    molecule.write("pdbqt", str(pdbqt_path), overwrite=True)

# 将蛋白质转换为PDBQT格式
protein_pdbqt_path = DATA / "protein.pdbqt"
pdb_to_pdbqt(protein_path, protein_pdbqt_path)

# 自动检测配体残基名称并计算口袋参数

# 定义标准残基名称（20种氨基酸和常见的溶剂、离子等）
standard_resnames = [
    'ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE',
    'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL',
    'HOH', 'WAT', 'SOL', 'NA', 'CL', 'K', 'CA', 'MG', 'ZN', 'FE', 'SO4',
    'PO4', 'CO', 'NAG', 'MAN', 'BMA'
]

# 选择非蛋白质和非标准残基（可能是配体）
non_protein = structure.select_atoms('not protein and not resname ' + ' '.join(standard_resnames))

# 获取潜在配体的唯一残基名称
ligand_resnames = set(non_protein.residues.resnames)

if not ligand_resnames:
    raise ValueError("在PDB文件中未找到配体残基。")

# 如果有多个配体，可以根据原子数选择最大的那个
ligand_residue_atoms_count = {}
for resname in ligand_resnames:
    ligand_atoms = structure.select_atoms(f'resname {resname}')
    ligand_residue_atoms_count[resname] = len(ligand_atoms)

# 选择原子数最多的残基名称作为配体
ligand_resname = max(ligand_residue_atoms_count, key=ligand_residue_atoms_count.get)
print(f"检测到的配体残基名称：{ligand_resname}")

# 选择配体原子
ligand = structure.select_atoms(f"resname {ligand_resname}")

# 计算口袋中心和大小
pocket_center = (ligand.positions.max(axis=0) + ligand.positions.min(axis=0)) / 2
pocket_size = ligand.positions.max(axis=0) - ligand.positions.min(axis=0) + 5

# 定义多个配体的SMILES
smiles_list = [
    # 在此处添加您的SMILES字符串列表
    "CC(C)S(=O)(=O)c1ccccc1Nc1nc(Nc2ccc(NC(=O)CN3CCC(C(=O)O)CC3)cc2)ncc1Cl",
    "CC(C)S(=O)(=O)c1ccccc1Nc1nc(Nc2ccc(CN3CCC(C(=O)NCCO)CC3)cc2)ncc1Cl",
    # 省略其他SMILES以节省空间
]

# 打开out_text.txt文件以保存输出
out_text_path = DATA / "out_text.txt"
with open(out_text_path, "w", encoding="utf-8") as out_file:
    # 遍历每个配体
    for idx, smiles in enumerate(smiles_list):
        ligand_idx = idx + 1
        ligand_name = f"ligand_{ligand_idx}"
        docking_pose_name = f"docking_poses_{ligand_idx}"
        ligand_pdbqt_path = DATA / f"{ligand_name}.pdbqt"
        docking_sdf_path = DATA / f"{docking_pose_name}.sdf"

        # 将SMILES转换为PDBQT格式
        molecule = pybel.readstring("smi", smiles)
        molecule.OBMol.CorrectForPH(7.4)
        molecule.addh()
        molecule.make3D(forcefield="mmff94s", steps=10000)
        for atom in molecule.atoms:
            atom.OBAtom.GetPartialCharge()
        molecule.write("pdbqt", str(ligand_pdbqt_path), overwrite=True)

        # 写入配体和对接文件的对应关系
        out_file.write(f"SMILES {ligand_idx}: {smiles}\n")
        out_file.write(f"配体文件: {ligand_name}.pdbqt\n")
        out_file.write(f"对接结果文件: {docking_pose_name}.sdf\n\n")

        # 运行Smina对接
        result = subprocess.run(
            [
                "smina",
                "--ligand", str(ligand_pdbqt_path),
                "--receptor", str(protein_pdbqt_path),
                "--center_x", str(pocket_center[0]),
                "--center_y", str(pocket_center[1]),
                "--center_z", str(pocket_center[2]),
                "--size_x", str(pocket_size[0]),
                "--size_y", str(pocket_size[1]),
                "--size_z", str(pocket_size[2]),
                "--out", str(docking_sdf_path),
                "--exhaustiveness", "8"
            ],
            capture_output=True,
            text=True
        )

        # 将Smina输出写入文件
        out_file.write(f"Smina输出 (配体 {ligand_idx}):\n")
        out_file.write(result.stdout)
        out_file.write("\n" + "="*80 + "\n\n")

    # 在文件末尾添加所有配体与对接结果的对应关系
    out_file.write("配体与对接结果对应关系:\n")
    for idx in range(len(smiles_list)):
        ligand_idx = idx + 1
        ligand_name = f"ligand_{ligand_idx}"
        docking_pose_name = f"docking_poses_{ligand_idx}"
        out_file.write(f"{ligand_name}.pdbqt 对接到 {docking_pose_name}.sdf\n")




*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /Users/wangyang/Desktop/teachopencadd/test/data/protein.pdb)



检测到的配体残基名称：IRE


In [18]:

# 定义分割SDF文件的函数
def split_sdf_file(sdf_path):
    """
    将SDF文件拆分为每个分子单独的文件。
    每个文件以连续的数字命名。

    参数
    ----------
    sdf_path: str 或 pathlib.Path
        要拆分的SDF文件路径。
    """
    sdf_path = Path(sdf_path)
    stem = sdf_path.stem
    parent = sdf_path.parent
    molecules = pybel.readfile("sdf", str(sdf_path))
    for i, molecule in enumerate(molecules, 1):
        molecule.write("sdf", str(parent / f"{stem}_{i}.sdf"), overwrite=True)
    return

# 分割每个对接结果的SDF文件
for idx in range(len(smiles_list)):
    split_sdf_file(DATA / f"docking_poses_{idx + 1}.sdf")


In [21]:
# 可视化（可选）
docking_pose_id = 2  # 选择要查看的对接结果编号
view = nv.show_structure_file(
    str(DATA / f"docking_poses_{docking_pose_id}.sdf"),
    representations=[{"params": {}, "type": "licorice"}],
)
view.add_pdbid("2ito")
view  # 显示可视化结果

NGLWidget()

# 分子对接

In [1]:
import warnings  # 导入警告模块
from pathlib import Path  # 导入路径处理模块
import subprocess  # 导入子进程模块
import numpy as np  # 导入NumPy模块
from MDAnalysis import Universe  # 从MDAnalysis导入Universe类
from openbabel import pybel  # 从Open Babel导入pybel模块
import nglview as nv  # 导入nglview用于可视化

# 忽略警告信息
warnings.filterwarnings("ignore")

class Structure(Universe):
    """用于加载结构的核心对象。"""

    @classmethod
    def from_string(cls, pdb_path):
        """从本地PDB文件加载结构。"""
        return cls(pdb_path)

# 定义路径
HERE = Path.cwd()
DATA = HERE / "data"
DATA.mkdir(exist_ok=True)

# 加载本地结构
pdb_path = DATA / "5i50.pdb"
structure = Structure.from_string(pdb_path)

# 准备蛋白质文件
protein_path = DATA / "protein.pdb"
protein = structure.select_atoms("protein")
protein.write(str(protein_path))

def pdb_to_pdbqt(pdb_path, pdbqt_path, pH=7.4):
    """将PDB文件转换为PDBQT文件。"""
    molecule = list(pybel.readfile("pdb", str(pdb_path)))[0]
    molecule.OBMol.CorrectForPH(pH)
    molecule.addh()
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()
    molecule.write("pdbqt", str(pdbqt_path), overwrite=True)

# 将蛋白质转换为PDBQT格式
protein_pdbqt_path = DATA / "protein.pdbqt"
pdb_to_pdbqt(protein_path, protein_pdbqt_path)

# 自动检测配体残基名称并计算口袋参数

# 定义标准残基名称（20种氨基酸和常见的溶剂、离子等）
standard_resnames = [
    'ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE',
    'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL',
    'HOH', 'WAT', 'SOL', 'NA', 'CL', 'K', 'CA', 'MG', 'ZN', 'FE', 'SO4',
    'PO4', 'CO', 'NAG', 'MAN', 'BMA'
]

# 选择非蛋白质和非标准残基（可能是配体）
non_protein = structure.select_atoms('not protein and not resname ' + ' '.join(standard_resnames))

# 获取潜在配体的唯一残基名称
ligand_resnames = set(non_protein.residues.resnames)

if not ligand_resnames:
    raise ValueError("在PDB文件中未找到配体残基。")

# 如果有多个配体，可以根据原子数选择最大的那个
ligand_residue_atoms_count = {}
for resname in ligand_resnames:
    ligand_atoms = structure.select_atoms(f'resname {resname}')
    ligand_residue_atoms_count[resname] = len(ligand_atoms)

# 选择原子数最多的残基名称作为配体
ligand_resname = max(ligand_residue_atoms_count, key=ligand_residue_atoms_count.get)
print(f"检测到的配体残基名称：{ligand_resname}")

# 选择配体原子
ligand = structure.select_atoms(f"resname {ligand_resname}")

# 计算口袋中心和大小
pocket_center = (ligand.positions.max(axis=0) + ligand.positions.min(axis=0)) / 2
pocket_size = ligand.positions.max(axis=0) - ligand.positions.min(axis=0) + 5

# 定义多个配体的SMILES
smiles_list = [
    # 在此处添加您的SMILES字符串列表
    "O=c1c(O)c(-c2ccc(O)c(O)c2)oc2cc(O)cc(O)c12",
]

# 打开out_text.txt文件以保存输出
out_text_path = DATA / "out_text.txt"
with open(out_text_path, "w", encoding="utf-8") as out_file:
    # 遍历每个配体
    for idx, smiles in enumerate(smiles_list):
        ligand_idx = idx + 1
        ligand_name = f"ligand_{ligand_idx}"
        docking_pose_name = f"docking_poses_{ligand_idx}"
        ligand_pdbqt_path = DATA / f"{ligand_name}.pdbqt"
        docking_sdf_path = DATA / f"{docking_pose_name}.sdf"

        # 将SMILES转换为PDBQT格式
        molecule = pybel.readstring("smi", smiles)
        molecule.OBMol.CorrectForPH(7.4)
        molecule.addh()
        molecule.make3D(forcefield="mmff94s", steps=10000)
        for atom in molecule.atoms:
            atom.OBAtom.GetPartialCharge()
        molecule.write("pdbqt", str(ligand_pdbqt_path), overwrite=True)

        # 写入配体和对接文件的对应关系
        out_file.write(f"SMILES {ligand_idx}: {smiles}\n")
        out_file.write(f"配体文件: {ligand_name}.pdbqt\n")
        out_file.write(f"对接结果文件: {docking_pose_name}.sdf\n\n")

        # 运行Smina对接，并显示进度
        print(f"开始对接配体 {ligand_idx}...")
        process = subprocess.Popen(
            [
                "smina",
                "--ligand", str(ligand_pdbqt_path),
                "--receptor", str(protein_pdbqt_path),
                "--center_x", str(pocket_center[0]),
                "--center_y", str(pocket_center[1]),
                "--center_z", str(pocket_center[2]),
                "--size_x", str(pocket_size[0]),
                "--size_y", str(pocket_size[1]),
                "--size_z", str(pocket_size[2]),
                "--out", str(docking_sdf_path),
                "--exhaustiveness", "8"
            ],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )
        # 实时读取输出
        smina_output = ''
        for line in process.stdout:
            print(line, end='')  # 显示进度
            out_file.write(line)  # 将输出写入文件
            smina_output += line
        process.wait()
        print(f"配体 {ligand_idx} 对接完成。\n")
        out_file.write("\n" + "="*80 + "\n\n")

    # 在文件末尾添加所有配体与对接结果的对应关系
    out_file.write("配体与对接结果对应关系:\n")
    for idx in range(len(smiles_list)):
        ligand_idx = idx + 1
        ligand_name = f"ligand_{ligand_idx}"
        docking_pose_name = f"docking_poses_{ligand_idx}"
        out_file.write(f"{ligand_name}.pdbqt 对接到 {docking_pose_name}.sdf\n")




*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /Users/wangyang/Desktop/AIDD/15_Mol2_to_Smiles/data/protein.pdb)



检测到的配体残基名称：DG
开始对接配体 1...
   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: 1789448577

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+---------

In [2]:
# 定义分割SDF文件的函数
def split_sdf_file(sdf_path):
    """
    将SDF文件拆分为每个分子单独的文件。
    每个文件以连续的数字命名。

    参数
    ----------
    sdf_path: str 或 pathlib.Path
        要拆分的SDF文件路径。
    """
    sdf_path = Path(sdf_path)
    stem = sdf_path.stem
    parent = sdf_path.parent
    molecules = pybel.readfile("sdf", str(sdf_path))
    for i, molecule in enumerate(molecules, 1):
        molecule.write("sdf", str(parent / f"{stem}_{i}.sdf"), overwrite=True)
    return

# 分割每个对接结果的SDF文件
for idx in range(len(smiles_list)):
    split_sdf_file(DATA / f"docking_poses_{idx + 1}.sdf")


In [4]:
# 可视化（可选）
docking_pose_id = 1  # 选择要查看的对接结果编号
view = nv.show_structure_file(
    str(DATA / f"docking_poses_{docking_pose_id}.sdf"),
    representations=[{"params": {}, "type": "licorice"}],
)
view.add_pdbid("2ito")
view  # 显示可视化结果

NGLWidget()